In [ ]:
import pandas as pd
from sklearn import linear_model

# Define relative file paths for training and testing data
train_file = "..\\Data\\Train\\trains1990s.csv"

# Load and format training data (only using PCEPI)
train_df = pd.read_csv(train_file, parse_dates=['observation_date'], date_format="%m/%y%").iloc[:,:2]
train_df['group'] = 0
train_df['time_idx'] = train_df.index

In [ ]:
numVals = train_df.shape[0]

valRatio = 0.2
splitLoc = int(numVals*(1-valRatio))

train = train_df.iloc[:splitLoc, :]
val = train_df.iloc[splitLoc:, :]

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
display(train.index.values.reshape(-1, 1).shape)
#linear model on just PCEPI
regr.fit(train.index.values.reshape(-1, 1), train['fred_PCEPI'])

In [ ]:
valDates = val.index.values.reshape(-1, 1)
y_hat = regr.predict(valDates)

display(y_hat)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(valDates, y_hat)
plt.plot(valDates, val['fred_PCEPI'])
plt.show()

In [ ]:
trainDates = train.index.values.reshape(-1, 1)

y_train_hat = regr.predict(trainDates)

plt.plot(trainDates, y_train_hat)
plt.plot(trainDates, train['fred_PCEPI'])
plt.show()

In [ ]:
import numpy as np
y_hats = np.concatenate((y_train_hat, y_hat))
display(y_hats.shape)

In [ ]:
plt.figure(figsize=(20, 10))
ax = plt.axes()
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.locator_params(axis='x', nbins=10)
plt.plot(train_df['observation_date'], y_hats)
plt.plot(train_df['observation_date'], train_df['fred_PCEPI'])
ax.fill_between(train_df['observation_date'][splitLoc:], train_df['fred_PCEPI'][0], y_hats[-1], facecolor='green', step='pre', alpha=0.5)
plt.title('Linear Regressor Predictions on PCEPI')
plt.xlabel('Date')
plt.ylabel('PCEPI')
plt.show()

In [ ]:
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

linearValR2 = r2_score(val['fred_PCEPI'], y_hat)
linearTrainR2 = r2_score(train['fred_PCEPI'], y_train_hat)
linearValRMSE = root_mean_squared_error(val['fred_PCEPI'], y_hat)
linearTrainRMSE = root_mean_squared_error(train['fred_PCEPI'], y_train_hat)
#display(linearValError, linearTrainError)
RMSEDf = pd.DataFrame(np.concatenate((np.array(['Linear']), np.array([linearTrainRMSE, linearValRMSE, linearTrainR2, linearValR2]))).reshape(1, -1), columns=['Model', 'Training RMSE', 'Validation RMSE', 'Training R2', 'Validation R2'])

In [ ]:
RMSEDf.to_csv('Linear_PCEPI_eval.csv')

### Pytorch Forecasting Linear Regression Model

In [ ]:
import torch
from typing import Dict
from pytorch_forecasting.models import BaseModel
from pytorch_forecasting.metrics.point import RMSE

class LinearRegressionModule(torch.nn.Module):
 
    def __init__(self, input_size : int, output_size : int):
        super(LinearRegressionModule, self).__init__()
        #a single 1-1 linear function
        self.linear = torch.nn.Linear(input_size, output_size)
 
    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [ ]:
class LinearRegressionModel(BaseModel):
    def __init__(self, input_size: int, output_size: int, **kwargs):
        # saves arguments in signature to `.hparams` attribute, mandatory call - do not skip this
        self.save_hyperparameters()
        # pass additional arguments to BaseModel.__init__, mandatory call - do not skip this
        super().__init__(loss=RMSE(), **kwargs)
        self.network = LinearRegressionModule(
            input_size=self.hparams.input_size,
            output_size=self.hparams.output_size
        )

    def forward(self, x: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        # x is a batch generated based on the TimeSeriesDataset
        network_input = x["encoder_cont"].squeeze(-1)
        prediction = self.network(network_input)

        # rescale predictions into target space
        prediction = self.transform_output(prediction, target_scale=x["target_scale"])

        # We need to return a dictionary that at least contains the prediction
        # The parameter can be directly forwarded from the input.
        # The conversion to a named tuple can be directly achieved with the `to_network_output` function.
        return self.to_network_output(prediction=prediction)

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet

encoderLength = 10
predictionLength = 10

trainDataset = TimeSeriesDataSet(
    train,
    group_ids=['group'],
    target='fred_PCEPI',
    time_idx='time_idx',
    min_encoder_length=encoderLength,
    max_encoder_length=encoderLength,
    min_prediction_length=predictionLength,
    max_prediction_length=predictionLength,
    time_varying_unknown_reals=['fred_PCEPI'],
)

valDataset = TimeSeriesDataSet.from_dataset(trainDataset, train_df, predict=True, stop_randomization=True)

In [ ]:
batch_size = 1
trainLoader = trainDataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
valLoader = valDataset.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

No categorical variables, one continuous variable, one continuous target variable. Normalise target values.

In [ ]:
model = LinearRegressionModel.from_dataset(trainDataset, input_size=encoderLength, output_size=predictionLength)

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")

#turned off logging, but by default this produces Tensorboard-interpretable logs every 50 steps
trainer = Trainer(fast_dev_run=False, callbacks=[early_stop_callback], logger=False)
trainer.fit(model, train_dataloaders=trainLoader, val_dataloaders=valLoader)

The above takes a long time to train a simple linear regressor.

In [ ]:
predictions = model.predict(
    valLoader, return_y=True, trainer_kwargs=dict(accelerator="cpu")
)
RMSE()(predictions.output, predictions.y)

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions = model.predict(
    valLoader, mode="raw", return_x=True, trainer_kwargs=dict(accelerator="cpu")
)
print(raw_predictions.x)

In [ ]:
model.plot_prediction(
    raw_predictions.x, raw_predictions.output, idx=0, add_loss_to_title=True
)

Seem to have got unlucky here?